In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from lightgbm import LGBMClassifier

from pyprojroot import here

In [ ]:
# Load the data
data = pd.read_csv(here() / 'data/preprocessed/targets-d.csv')

# Replace special characters in column names
data.columns = data.columns.str.replace('[{}:,"]', '-', regex=True)

# Convert the 'Datetime' column to datetime
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Extract features from the 'Datetime' column
data['Year'] = data['Datetime'].dt.year
data['Month'] = data['Datetime'].dt.month
data['Day'] = data['Datetime'].dt.day

# Now you can drop the original 'Datetime' column
data = data.drop('Datetime', axis=1)

# Get all columns with 'Target-' prefix
target_cols = data.filter(regex='^Target-', axis=1).columns

# Drop these columns
X = data.drop(target_cols, axis=1)
y = data['Target-Q90_bedoccupying']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create and train the LGBM classifier model with parameters
model = LGBMClassifier(
    boosting_type='dart',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=100,
    subsample_for_bin=200000,
    objective=None,
    class_weight=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None,
    n_jobs=-1,
    silent=True,
    importance_type='split'
)

# Fit the model to the training data
model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# Calculate precision, recall, and accuracy
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)
